# SDG-based Answer Organization and Summarization

## Overview

This notebook performs the following tasks:

1. **Sorting answers by SDGs**, organizing the files according to their associated Sustainable Development Goals.
2. **Updating citations** and generating a **summary for each SDG**, consolidating all questions related to that goal.

## Configuration

At the beginning of the notebook, update the **path variables** to specify the input data, output directories, and any other required resources.


In [3]:
# Folders
answers_folder = "./Results/Answers/Answers-Subtopics/Dev set/Updated_citations"
sdg_root_folder = "./Results/Questions/Test questions different prompts/4-Filtred questions with SDGs"
output_folder_answer = "./Results/Answers/Answers-SDGs"
output_path_summary = "./Results/Summaries/UniqueSummary-EachSDG"



# Sort answers File for SDGs 

In [2]:
import os
import json




# Make sure output folder exists
os.makedirs(output_folder_answer, exist_ok=True)

# Build a dictionary of all SDG files in all subfolders
sdg_files = {}
for root, _, files in os.walk(sdg_root_folder):
    for f in files:
        if f.endswith(".json"):
            sdg_files[f] = os.path.join(root, f)

# Iterate over all answer files
for answer_file_name in os.listdir(answers_folder):
    if not answer_file_name.endswith(".json"):
        continue
    
    answer_file_path = os.path.join(answers_folder, answer_file_name)
    
    # Check if output file already exists
    output_file_path = os.path.join(output_folder_answer, answer_file_name)
    if os.path.exists(output_file_path):
        print(f"Skipping {answer_file_name} (output already exists)")
        continue
    
    # Construct corresponding SDG file name
    sdg_file_name = answer_file_name.replace("answers_new_cit-answes-", "final_questions-SDGs-final_questions-")
    sdg_file_name_with_ext = sdg_file_name if sdg_file_name.endswith(".json") else sdg_file_name + ".json"
    
    if sdg_file_name_with_ext not in sdg_files:
        print(f"SDG file not found for {answer_file_name}")
        continue
    
    sdg_file_path = sdg_files[sdg_file_name_with_ext]
    
    # Load JSON data
    with open(answer_file_path, 'r', encoding='utf-8') as f:
        answers_data = json.load(f)
    
    with open(sdg_file_path, 'r', encoding='utf-8') as f:
        sdg_data = json.load(f)
    
    # Flatten SDG data into question -> SDG list
    question_to_sdgs = {}
    for cluster_id, questions in sdg_data.items():
        for q in questions:
            question_to_sdgs[q["question"]] = q["sdg_scores"]
    
    # Prepare SDG grouped output (only create keys for SDGs with answers)
    sdg_grouped = {}
    
    for item in answers_data:
        question = item.get("question")
        if question not in question_to_sdgs:
            continue
        sdg_scores = question_to_sdgs[question]
        # For each SDG index with score 1, add this answer
        for sdg_index, score in enumerate(sdg_scores):
            if score:
                sdg_key = f"sdg-{sdg_index + 1}"  # <-- index+1 for 1-based SDG numbering
                if sdg_key not in sdg_grouped:
                    sdg_grouped[sdg_key] = []
                sdg_grouped[sdg_key].append(item)
    
    # Save output
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(sdg_grouped, f, indent=2, ensure_ascii=False)

    print(f"Processed and saved: {output_file_path}")

Skipping answers_new_cit-answes-Haiti_Gang violence and humanitarian crisis in Haiti-Week 40 2024-prompt-1.json (output already exists)
Skipping answers_new_cit-answes-Nigeria_Flooding in Nigeria-Week 37 2024-prompt-1.json (output already exists)
Skipping answers_new_cit-answes-United Kingdom_UK riots-Week 32 2024-prompt-1.json (output already exists)
Skipping answers_new_cit-answes-Sudan_Sudan conflict-Week 34 2024-prompt-1.json (output already exists)
Processed and saved: ./Results/Answers/Answers-SDGs/answers_new_cit-answes-Jamaica_Hurricane Beryl-Week 28 2024-prompt-1.json
Skipping answers_new_cit-answes-Afghanistan_Afghanistan Floods-Week 21 2024-prompt-1.json (output already exists)
Skipping answers_new_cit-answes-Bangladesh_Cyclone Remal-Week 21 2024-prompt-1.json (output already exists)
Skipping answers_new_cit-answes-Indonesia_Floods and volcanic activity in Indonesia-Week 20 2024-prompt-1.json (output already exists)
Skipping answers_new_cit-answes-Ukraine_Ukraine-Week 23 202

# Create summary for each SDGs

In [ ]:
import os
import re
import json
import numpy as np
from openai import OpenAI

# Input and output paths


# OpenAI client
client = OpenAI(api_key=" ")

# Prompt template
prompt_template = """
Your task is to integrate the following pieces of text into a single, cohesive, and flowing narrative. The goal is to present as much of the original information as possible, not to summarize it briefly.

The text contains information with citations, formatted as `[number]`. It is crucial that you adhere to the following rules:

1. **Integrate all key information**: Combine sentences and ideas from the input to form a comprehensive and coherent text. Aim to include a good portion, if not all, of the provided details.
2. **Maintain original citations**: Every piece of information you include in the integrated text must retain its original citation(s).
3. **Handle citations when combining**: If you rephrase or combine sentences containing information from multiple sources, ensure that *all* relevant original citation numbers for that combined information are included at the end of the new sentence. For example, if a new sentence merges details from original sentences cited `[1]` and `[5]`, the new sentence should be followed by `[1][5]`.
4. **Ensure logical flow**: Arrange the information in a way that creates a natural and readable progression of ideas, even if it means reordering content from the original input.
5. **Avoid external knowledge**: Your output must be based *solely* on the information provided in the input text. Do not introduce any outside facts or personal opinions.

Here is the text to integrate:
{text_to_integrate}
"""

# Ensure output folder exists
os.makedirs(output_path_summary, exist_ok=True)

# Collect all input files
try:
    json_files = np.sort([f for f in os.listdir(output_folder_answer) if f.endswith('.json')])
except FileNotFoundError:
    print(f"Error: The directory '{output_folder_answer}' was not found.")
    json_files = []

# Process each SDG JSON file
for file in json_files:
    file_name = file.replace('answers_new_cit-answes-', '').replace('-prompt-1.json', '')
    
    # Check if output file already exists
    output_filename = os.path.join(output_path_summary, f"summary_{file_name}.json")
    if os.path.exists(output_filename):
        print(f"--- Skipping file: {file_name} (output already exists) ---\n")
        continue
    
    print(f"--- Processing file: {file_name} ---")

    try:
        with open(os.path.join(output_folder_answer, file), 'r') as f:
            json_data = json.load(f)
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from file: {file}")
        continue
    except Exception as e:
        print(f"An error occurred while reading {file}: {e}")
        continue

    # Group answers by SDG key
    sdg_groups = {}
    for sdg_key, answers in json_data.items():
        sdg_groups[sdg_key] = [a for a in answers if a]  # Keep full item for context tracking

    # Process each SDG group
    modified_answers = {}
    used_contexts = {}

    for sdg_key, answers_in_group in sdg_groups.items():
        modified_answers[sdg_key] = []
        used_contexts[sdg_key] = {}

        for i, item in enumerate(answers_in_group):
            original_answer = item.get("updated_retrieved_answer", "")
            offset = i * 10
            modified_answer = original_answer

            citations_found = set(re.findall(r'\[(\d+)\]', original_answer))
            sorted_citations = sorted([int(c) for c in citations_found], reverse=True)

            for old_citation_num in sorted_citations:
                new_citation_num = old_citation_num + offset
                modified_answer = re.sub(
                    r'\[{}\]'.format(re.escape(str(old_citation_num))),
                    '[{}]'.format(new_citation_num),
                    modified_answer
                )

                # Track context if available
                if "new_citations" in item and old_citation_num in item["new_citations"]:
                    idx = item["new_citations"].index(old_citation_num)
                    ctx = item.get("new_used_contexts", [])[idx]
                    used_contexts[sdg_key][new_citation_num] = ctx

            modified_answers[sdg_key].append(modified_answer)

    # Generate summaries for each SDG group
    final_output = {}

    for sdg_key, answers in modified_answers.items():
        if not answers:
            continue
        text_to_summarize = "\n\n".join(answers)
        custom_prompt = prompt_template.format(text_to_integrate=text_to_summarize)

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant specialized in combining text."},
                    {"role": "user", "content": custom_prompt}
                ],
                temperature=0.0
            )
            summary = response.choices[0].message.content.strip()
            final_output[sdg_key] = {
                "summary": summary,
                "used_contexts": used_contexts[sdg_key]
            }
            print(f"✅ Summarized {sdg_key} for {file_name}")

        except Exception as e:
            error_message = f"Error: Could not summarize due to {e}"
            final_output[sdg_key] = {
                "summary": error_message,
                "used_contexts": used_contexts[sdg_key]
            }
            print(f"❌ Error summarizing {sdg_key} for {file_name}: {e}")

    # Save the summarized answers to a new JSON file
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(final_output, f, indent=2, ensure_ascii=False)
        print(f"💾 Saved summary for {file_name} to {output_filename}\n")
    except Exception as e:
        print(f"❌ Could not write to file {output_filename}: {e}\n")

--- Skipping file: Afghanistan_Afghanistan Floods-Week 21 2024 (output already exists) ---

--- Skipping file: Bangladesh_Cyclone Remal-Week 21 2024 (output already exists) ---

--- Skipping file: Haiti_Gang violence and humanitarian crisis in Haiti-Week 40 2024 (output already exists) ---

--- Skipping file: India_LandslideFloods-Week 31 2024 (output already exists) ---

--- Skipping file: Indonesia_Floods and volcanic activity in Indonesia-Week 20 2024 (output already exists) ---

--- Skipping file: Israel_Israel-Hamas war-Week 19 2024 (output already exists) ---

--- Processing file: Israel_Israel_Palestine_confilct-Week 40 2024 ---
✅ Summarized sdg-3 for Israel_Israel_Palestine_confilct-Week 40 2024
✅ Summarized sdg-16 for Israel_Israel_Palestine_confilct-Week 40 2024
✅ Summarized sdg-2 for Israel_Israel_Palestine_confilct-Week 40 2024
✅ Summarized sdg-9 for Israel_Israel_Palestine_confilct-Week 40 2024
✅ Summarized sdg-15 for Israel_Israel_Palestine_confilct-Week 40 2024
✅ Summari